In [1]:
import pandas as pd
import numpy as np
import os, sys, gc

In [2]:
path = './t-academy-data/'

In [3]:
books = pd.read_csv(path+"books.csv")
book_tags = pd.read_csv(path+"book_tags.csv")
train = pd.read_csv(path+"train.csv")
test = pd.read_csv(path+"test.csv")
tags = pd.read_csv(path+"tags.csv")
to_read = pd.read_csv(path+"to_read.csv")

In [4]:
train['book_id'] = train['book_id'].astype(str)
test['book_id'] = test['book_id'].astype(str)
books['book_id'] = books['book_id'].astype(str)

In [5]:
popular_rec_model = books.sort_values(by='books_count', ascending=False)['book_id'].values[:500]

In [6]:
sol = test.groupby(['user_id'])['book_id'].agg({'unique'}).reset_index() 

In [7]:
gt = {}
for user in sol['user_id'].unique():
    gt[user] = sol[sol['user_id'] == user]['unique'].values[0].tolist()

In [8]:
rec_df = pd.DataFrame()
rec_df['user_id'] = train['user_id'].unique()

In [9]:
user2idx = {}
for i, user in enumerate(train['user_id'].unique()):
    user2idx[user] = i

book2idx = {}
for i, book in enumerate(train['book_id'].unique()):
    book2idx[book] = i

In [10]:
idx2user = {i: user for user, i in user2idx.items()}
idx2book = {i: book for book, i in book2idx.items()}

In [11]:
data = train[['user_id', 'book_id']].reset_index(drop=True)
useridx = data['user_id'] = train['user_id'].apply(lambda x: user2idx[x]).values
bookidx = data['book_id'] = train['book_id'].apply(lambda x: book2idx[x]).values
rating = np.ones(len(data))

In [12]:
from scipy import sparse

purchase_sparse = sparse.csr_matrix((rating, (useridx, bookidx)), shape=(len(set(useridx)), len(set(bookidx))))

In [13]:
R = purchase_sparse.toarray()

# ALS 협업필터링

In [14]:
from implicit.evaluation import *
from implicit.als import AlternatingLeastSquares as ALS
from implicit.bpr import BayesianPersonalizedRanking as BPR

C:\Users\user\anaconda3\envs\rec-sys\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
als_model = ALS(factors=20, regularization=0.01, iterations=100)
als_model.fit(purchase_sparse.T)

100%|██████████| 100/100 [00:55<00:00,  1.79it/s]


In [21]:
als_model.recommend(0, purchase_sparse, N=200)

ValueError: user_items must contain 1 row for every user in userids

In [24]:
purchase_sparse

<53382x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 386446 stored elements in Compressed Sparse Row format>